In [1]:
import numpy as np
from clean_scripts import *
import scipy
import sys
import pickle
import pandas as pd

sys.path.append('clean_scripts')
import player_shotInfo2

/Users/geoffreypoon/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
bins, binRange = ([25,18], [[-250,250], [-47.5,312.5]])

hist2d, xedges, yedges, binnumber = scipy.stats.binned_statistic_2d([0], [0], 
                                                                    [0],
                                                                    statistic='sum',
                                                                    bins=bins, 
                                                                    range=binRange)

binDat = (bins, binRange, xedges, yedges, binnumber)
X_T, Y_T = np.meshgrid(xedges[:-1] + 10, yedges[:-1] + 10)
X, Y = (X_T.T, Y_T.T)
# X, Y = np.meshgrid(xedges[:-1], yedges[:-1])

In [3]:
X.shape

(25, 18)

In [4]:
print(xedges[-1]-xedges[0], yedges[-1]-yedges[0])

500.0 360.0


In [5]:
# W_norm_att = np.loadtxt('clean_scripts/W_norm_SHOT_ATTEMPTED_FLAG.txt')
# H_norm_att = np.loadtxt('clean_scripts/H_norm_SHOT_ATTEMPTED_FLAG.txt')

# W_norm_made = np.loadtxt('clean_scripts/W_norm_SHOT_MADE_FLAG.txt')
# H_norm_made = np.loadtxt('clean_scripts/H_norm_SHOT_MADE_FLAG.txt')

# with open ('clean_scripts/top_players_nameList.pickle', 'rb') as fp:
#     top_players_nameList = pickle.load(fp)

In [6]:
import os
import simplejson as json
from clean_scripts import nmf_func as nmf


season = '2016-17'
phi = 30

######################

shots_path = os.path.abspath(os.getcwd()) + '/data/shots/shots_data_%s.json'%season
with open(shots_path, 'r') as json_data:
    d = json.load(json_data)
    json_data.close()

shots_dat = pd.DataFrame(d['resultSets'][0]['rowSet'], 
                         columns=d['resultSets'][0]['headers'])

df = pd.DataFrame(shots_dat, 
                  columns = ['PLAYER_ID', 'PLAYER_NAME', 
                             'TEAM_ID', 'TEAM_NAME', 'ACTION_TYPE',
                             'SHOT_DISTANCE', 'SHOT_TYPE', 
                             'LOC_X', 'LOC_Y', 
                             'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG'])

num_players = 300
top_players_shotNum = df.PLAYER_NAME.value_counts()[:num_players]
top_players_nameList = top_players_shotNum.index.tolist()




W_norm, H_norm = (dict(), dict())
for flag_name in ['SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG']:
    directory = 'clean_scripts/%s/%s/shotHist_LGCP_phi%d_seed546682'%(season, flag_name, phi)
    LL = np.zeros((len(top_players_nameList), np.prod(bins)))
    for i, player in enumerate(top_players_nameList):
        lambdaN_v = np.loadtxt(directory + '/lambda_%s.txt'%(player))
        norm_lambdaN_v = lambdaN_v / np.sum(lambdaN_v)
        LL[i,:] = norm_lambdaN_v[:]

    n_features = 10
    W_norm[flag_name], H_norm[flag_name] = nmf.run(LL, n_features=n_features)

    
######################
    
player = 'Stephen Curry'
player_index = top_players_nameList.index(player)
print(player_index, top_players_nameList[player_index])


directory = 'clean_scripts/%s/%s/shotHist_LGCP_phi30_seed546682/'
shot_made_src = (directory + 'lambda_%s.txt') %(season, 'SHOT_MADE_FLAG', player)
shot_att_src = (directory + 'lambda_%s.txt') %(season, 'SHOT_ATTEMPTED_FLAG', player)

shots_made_map = np.loadtxt(shot_made_src)
shots_att_map = np.loadtxt(shot_att_src)

tot_shots_made = shots_made_map.sum()
tot_shots_att = shots_att_map.sum()

print('Predicted FG%: ', tot_shots_made / tot_shots_att)
    
######################

nmf_made_map = np.dot(W_norm['SHOT_MADE_FLAG'], H_norm['SHOT_MADE_FLAG'])[player_index, :]
nmf_att_map = np.dot(W_norm['SHOT_ATTEMPTED_FLAG'], H_norm['SHOT_ATTEMPTED_FLAG'])[player_index, :]

nmf_FGper_map = (tot_shots_made / tot_shots_att) * (nmf_made_map / nmf_att_map)

9 Stephen Curry
Predicted FG%:  0.4797179150904743


In [56]:
def create_dat_arr(X, Y, fg_vec, att_vec, bottom_frac=0.02):
    def calc_threshold_att_prob(att_vec, bottom_frac):
        sorted_att_prob = np.array(sorted(att_vec, reverse=True))
        sorted_att_prob_cumsum = np.cumsum(sorted_att_prob)
        filtered_att_prob = sorted_att_prob[1-sorted_att_prob_cumsum >= bottom_frac]
        print(filtered_att_prob[-1])
        return filtered_att_prob[-1]
    
    def mask_vec(vec, att_vec, bottom_frac):
        thresh = calc_threshold_att_prob(att_vec, bottom_frac)
        masked_vec = np.copy(vec)
        masked_vec[att_vec < thresh] = float('nan')
        return masked_vec
    
    masked_att_vec = mask_vec(att_vec, att_vec, bottom_frac)
    masked_fg_vec = mask_vec(fg_vec, att_vec, bottom_frac)
    arr = np.hstack((
        X.reshape(-1,1), Y.reshape(-1,1), 
        masked_att_vec.reshape(-1,1), masked_fg_vec.reshape(-1,1)
    ))
    arr[:,2:] = arr[:,2:]*100
    return arr

arr = create_dat_arr(X, Y, nmf_FGper_map, nmf_att_map, bottom_frac=0.05)

0.0007187004398642233
0.0007187004398642233


In [57]:
df = pd.DataFrame(arr, columns=['X', 'Y', 'FGA_perc', 'FGM_perc'])
df

,X,Y,FGA_perc,FGM_perc
0,-240.0,-37.5,NaN,NaN
1,-240.0,-17.5,0.435436,35.636364
2,-240.0,2.5,1.174980,40.239643
3,-240.0,22.5,0.678524,51.521225
4,-240.0,42.5,0.394342,46.704212
5,-240.0,62.5,0.258554,30.591402
6,-240.0,82.5,0.131026,40.544102
7,-240.0,102.5,0.124381,45.468493
8,-240.0,122.5,0.148098,46.978854
9,-240.0,142.5,NaN,NaN


In [9]:
np.sum(df.FGA_perc)

94.96038456298204

In [10]:
arr = create_dat_arr(X, Y, nmf_FGper_map, nmf_att_map, bottom_frac=0.05)
df = pd.DataFrame(arr, columns=['X', 'Y', 'FGA_perc', 'FGM_perc'])
df_noNAN = df[np.isfinite(df.FGA_perc)]

0.0007187004398642233
0.0007187004398642233


In [11]:
df_noNAN.FGA_perc.max()

8.14976242875996

In [66]:
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
)
from bokeh.plotting import figure
import bokeh.palettes

arr = create_dat_arr(X, Y, nmf_FGper_map, nmf_att_map, bottom_frac=0.05)
df = pd.DataFrame(arr, columns=['X', 'Y', 'FGA_perc', 'FGM_perc'])
df_noNAN = df[np.isfinite(df.FGA_perc)]

# this is the colormap
colors = bokeh.palettes.magma(256)[::-1]
mapper = LinearColorMapper(palette = colors, nan_color = 'white',
                           low = 0., high = int(df_noNAN.FGA_perc.max()+1))

source = ColumnDataSource(df)

TOOLS = "hover,save,reset,wheel_zoom"

p1 = figure(x_range=(-250,250), y_range=(-47.5,470-47.5),
           plot_width=600, plot_height=660,
           tools=TOOLS, toolbar_location='below'
          )

p1.grid.grid_line_color = None
p1.axis.axis_line_color = None
p1.axis.major_tick_line_color = None
p1.axis.minor_tick_line_color = None
p1.axis.major_label_text_font_size = "0pt"
p1.axis.major_label_standoff = 0

# p1.image_url(url=['nba_court.jpg'], 
#             x=-250, y=-47.5,
#             w=500, h=470-47, anchor="bottom_left"
#            )

alpha=0.7

p1.rect(x="X", y="Y", width=20, height=20,
       source=source,
       fill_color={'field': 'FGA_perc', 'transform': mapper},
       fill_alpha=alpha,
       line_color=None,
       name="heatmap")



#######################################

lw = 2.5
color = 'black'

# Create the basketball hoop
# Diameter of a hoop is 18" so it has a radius of 9", which is a value
# 7.5 in our coordinate system
p1.circle([0], [0], size=2*7.5, 
          line_width=lw, line_color=color, fill_color=None)

# Create backboard
p1.rect([0], [-7.5], width=60, height=0.4, 
        line_width=lw, line_color=color, fill_color=None)

# The paint
# Create the outer box 0f the paint, width=16ft, height=19ft
p1.rect([0], [47.5], width=160, height=190, 
        line_width=lw, line_color=color,
        fill_color=None)
# Create the inner box of the paint, widt=12ft, height=19ft
p1.rect([0], [47.5], width=120, height=190, 
        line_width=lw, line_color=color,
        fill_color=None)

# Create free throw top arc
p1.circle([0], [142.5], size=140, 
          line_width=lw, line_color=color, fill_color=None)

# Restricted Zone, it is an arc with 4ft radius from center of the hoop
p1.arc([0], [0], 40, 0, np.pi, 
       line_width=lw, line_color=color)

# Three point line
# Create the side 3pt lines, they are 14ft long before they begin to arc
p1.line([-220, -220], [-47.5, 92.5],
        line_width=lw, line_color=color)
p1.line([220, 220], [-47.5, 92.5], 
        line_width=lw, line_color=color)
# 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
# I just played around with the theta values until they lined up with the 
# threes
p1.arc([0], [0], 239, np.pi*23/180, np.pi*157/180, 
       line_width=lw, line_color=color)

# Center Court
p1.arc([0], [422.5], 60, np.pi, 0,
       line_width=lw, line_color=color)
p1.arc([0], [422.5], 20, np.pi, 0,
       line_width=lw, line_color=color)

#######################################


color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="10pt",
                     ticker=BasicTicker(desired_num_ticks=10),
                     formatter=PrintfTickFormatter(format="%.1f%%"),
                     label_standoff=6, border_line_color=None, location=(0, 0), 
                     orientation="horizontal",
                     scale_alpha=alpha
                    )
p1.add_layout(color_bar, 'above')

p1.select_one(HoverTool).names = ["heatmap"]
p1.select_one(HoverTool).tooltips = [
    ('(X, Y)', '(@X, @Y)'),
    ('FGA% (rel. avg.)', '@FGA_perc%'),
    ('FG% (rel. avg.)', '@FGM_perc%'),
]

# show(p1)      # show the plot

0.0007187004398642233
0.0007187004398642233


In [65]:
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
)
from bokeh.plotting import figure
import bokeh.palettes
import matplotlib.cm as cm
import matplotlib as plt

arr = create_dat_arr(X, Y, nmf_FGper_map, nmf_att_map, bottom_frac=0.05)
df = pd.DataFrame(arr, columns=['X', 'Y', 'FGA_perc', 'FGM_perc'])
df_noNAN = df[np.isfinite(df.FGA_perc)]

# this is the colormap
# colors = bokeh.palettes.coolwarm(256)[::-1]
colormap =cm.get_cmap("RdYlBu") #choose any matplotlib colormap here
colors = [plt.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))][::-1]
# mapper = CategoricalColorMapper(
#     palette=[
#         LinearColorMapper(palette=colors, low=0., high=int(df_noNAN.FGM_perc.max()+1)), 
#         "blue"
#     ], factors=["foo", "bar"]
# )
mapper = LinearColorMapper(palette = colors, nan_color = 'white',
                           low = 0., high = int(df_noNAN.FGM_perc.max()+1))

source = ColumnDataSource(df)

TOOLS = "hover,save,reset,wheel_zoom"

p2 = figure(x_range=(-250,250), y_range=(-47.5,470-47.5),
           plot_width=600, plot_height=660,
           tools=TOOLS, toolbar_location='below'
          )

p2.grid.grid_line_color = None
p2.axis.axis_line_color = None
p2.axis.major_tick_line_color = None
p2.axis.minor_tick_line_color = None
p2.axis.major_label_text_font_size = "0pt"
p2.axis.major_label_standoff = 0

alpha=0.7

p2.rect(x="X", y="Y", width=20, height=20,
       source=source,
       fill_color={'field': 'FGM_perc', 'transform': mapper},
       fill_alpha=alpha,
       line_color=None,
       name="heatmap"
       )

# p2.image_url(url=['nba_court.jpg'], 
#             x=-250, y=-47.5,
#             w=600, h=660, anchor="bottom_left"
# )

#######################################

lw = 2.5
color = 'black'

# Create the basketball hoop
# Diameter of a hoop is 18" so it has a radius of 9", which is a value
# 7.5 in our coordinate system
p2.circle([0], [0], size=2*7.5, 
          line_width=lw, line_color=color, fill_color=None)

# Create backboard
p2.rect([0], [-7.5], width=60, height=0.4, 
        line_width=lw, line_color=color, fill_color=None)

# The paint
# Create the outer box 0f the paint, width=16ft, height=19ft
p2.rect([0], [47.5], width=160, height=190, 
        line_width=lw, line_color=color,
        fill_color=None)
# Create the inner box of the paint, widt=12ft, height=19ft
p2.rect([0], [47.5], width=120, height=190, 
        line_width=lw, line_color=color,
        fill_color=None)

# Create free throw top arc
p2.circle([0], [142.5], size=140, 
          line_width=lw, line_color=color, fill_color=None)

# Restricted Zone, it is an arc with 4ft radius from center of the hoop
p2.arc([0], [0], 40, 0, np.pi, 
       line_width=lw, line_color=color)

# Three point line
# Create the side 3pt lines, they are 14ft long before they begin to arc
p2.line([-220, -220], [-47.5, 92.5],
        line_width=lw, line_color=color)
p2.line([220, 220], [-47.5, 92.5], 
        line_width=lw, line_color=color)
# 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
# I just played around with the theta values until they lined up with the 
# threes
p2.arc([0], [0], 239, np.pi*23/180, np.pi*157/180, 
       line_width=lw, line_color=color)

# Center Court
p2.arc([0], [422.5], 60, np.pi, 0,
       line_width=lw, line_color=color)
p2.arc([0], [422.5], 20, np.pi, 0,
       line_width=lw, line_color=color)

#######################################


# p2.line(x=[0,250], y=[0,300])

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="10pt",
                     ticker=BasicTicker(desired_num_ticks=10),
                     formatter=PrintfTickFormatter(format="%d%%"),
                     label_standoff=6, border_line_color=None, location=(0, 0), 
                     orientation="horizontal",
                     scale_alpha=alpha
                    )
p2.add_layout(color_bar, 'above')

p2.select_one(HoverTool).names = ["heatmap"]
p2.select_one(HoverTool).tooltips = [
    ('(X, Y)', '(@X, @Y)'),
    ('FGA% (rel. avg.)', '@FGA_perc%'),
    ('FG% (rel. avg.)', '@FGM_perc%'),
]

# show(p2)      # show the plot

0.0007187004398642233
0.0007187004398642233


In [67]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure

# output_file('shotChart.html')

tab1 = Panel(child=p1, title="Percentage of Field Goal Attempts (FGA%)")
tab2 = Panel(child=p2, title="Field Goal Percentage (FG%)")
tabs = Tabs(tabs=[ tab1, tab2 ])

show(tabs)

In [15]:
# from bokeh.models.widgets import Panel, Tabs
# from bokeh.io import output_file, show
# from bokeh.plotting import figure

# from bokeh.plotting import figure
# from bokeh.embed import components

# tab1 = Panel(child=p1, title="Percentage of Field Goal Attempts (FGA%)")
# tab2 = Panel(child=p2, title="Field Goal Percentage (FG%)")
# tabs = Tabs(tabs=[ tab1, tab2 ])

# script, div = components(tabs)

In [16]:
# print(script)

In [17]:
# print(div)

In [18]:
from bokeh.models.widgets import TextInput
cast = TextInput(title="Cast names contains")
show(cast)
print(cast.value)

In [19]:
cast.value

''

In [20]:
from bokeh.plotting import show, output_notebook, output_file
from bokeh.models.widgets import AutocompleteInput

output_notebook()
# output_file('autocomplete.html')

input_widget = AutocompleteInput(completions=['pok', 'wer', 'foo', 'bar', 'baz'], title='test')
show(input_widget)

Loading BokehJS ...

In [21]:
input_widget.value

''

In [22]:
# bokeh.sampledata.download()

In [23]:

# from os.path import dirname, join

# import numpy as np
# import pandas.io.sql as psql
# import sqlite3 as sql

# from bokeh.plotting import figure
# from bokeh.layouts import layout, widgetbox
# from bokeh.models import ColumnDataSource, HoverTool, Div
# from bokeh.models.widgets import Slider, Select, TextInput
# from bokeh.io import curdoc
# from bokeh.sampledata.movies_data import movie_path

# conn = sql.connect(movie_path)
# query = open(join(dirname(__file__), 'query.sql')).read()
# movies = psql.read_sql(query, conn)

# movies["color"] = np.where(movies["Oscars"] > 0, "orange", "grey")
# movies["alpha"] = np.where(movies["Oscars"] > 0, 0.9, 0.25)
# movies.fillna(0, inplace=True)  # just replace missing values with zero
# movies["revenue"] = movies.BoxOffice.apply(lambda x: '{:,d}'.format(int(x)))

# with open(join(dirname(__file__), "razzies-clean.csv")) as f:
#     razzies = f.read().splitlines()
# movies.loc[movies.imdbID.isin(razzies), "color"] = "purple"
# movies.loc[movies.imdbID.isin(razzies), "alpha"] = 0.9

# axis_map = {
#     "Tomato Meter": "Meter",
#     "Numeric Rating": "numericRating",
#     "Number of Reviews": "Reviews",
#     "Box Office (dollars)": "BoxOffice",
#     "Length (minutes)": "Runtime",
#     "Year": "Year",
# }

# desc = Div(text=open(join(dirname(__file__), "description.html")).read(), width=800)

# # Create Input controls
# reviews = Slider(title="Minimum number of reviews", value=80, start=10, end=300, step=10)
# min_year = Slider(title="Year released", start=1940, end=2014, value=1970, step=1)
# max_year = Slider(title="End Year released", start=1940, end=2014, value=2014, step=1)
# oscars = Slider(title="Minimum number of Oscar wins", start=0, end=4, value=0, step=1)
# boxoffice = Slider(title="Dollars at Box Office (millions)", start=0, end=800, value=0, step=1)
# genre = Select(title="Genre", value="All",
#                options=open(join(dirname(__file__), 'genres.txt')).read().split())
# director = TextInput(title="Director name contains")
# cast = TextInput(title="Cast names contains")
# x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="Tomato Meter")
# y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="Number of Reviews")

# # Create Column Data Source that will be used by the plot
# source = ColumnDataSource(data=dict(x=[], y=[], color=[], title=[], year=[], revenue=[], alpha=[]))

# hover = HoverTool(tooltips=[
#     ("Title", "@title"),
#     ("Year", "@year"),
#     ("$", "@revenue")
# ])

# p = figure(plot_height=600, plot_width=700, title="", toolbar_location=None, tools=[hover])
# p.circle(x="x", y="y", source=source, size=7, color="color", line_color=None, fill_alpha="alpha")


# def select_movies():
#     genre_val = genre.value
#     director_val = director.value.strip()
#     cast_val = cast.value.strip()
#     selected = movies[
#         (movies.Reviews >= reviews.value) &
#         (movies.BoxOffice >= (boxoffice.value * 1e6)) &
#         (movies.Year >= min_year.value) &
#         (movies.Year <= max_year.value) &
#         (movies.Oscars >= oscars.value)
#     ]
#     if (genre_val != "All"):
#         selected = selected[selected.Genre.str.contains(genre_val)==True]
#     if (director_val != ""):
#         selected = selected[selected.Director.str.contains(director_val)==True]
#     if (cast_val != ""):
#         selected = selected[selected.Cast.str.contains(cast_val)==True]
#     return selected


# def update():
#     df = select_movies()
#     x_name = axis_map[x_axis.value]
#     y_name = axis_map[y_axis.value]

#     p.xaxis.axis_label = x_axis.value
#     p.yaxis.axis_label = y_axis.value
#     p.title.text = "%d movies selected" % len(df)
#     source.data = dict(
#         x=df[x_name],
#         y=df[y_name],
#         color=df["color"],
#         title=df["Title"],
#         year=df["Year"],
#         revenue=df["revenue"],
#         alpha=df["alpha"],
#     )

# controls = [reviews, boxoffice, genre, min_year, max_year, oscars, director, cast, x_axis, y_axis]
# for control in controls:
#     control.on_change('value', lambda attr, old, new: update())

# sizing_mode = 'fixed'  # 'scale_width' also looks nice with this example

# inputs = widgetbox(*controls, sizing_mode=sizing_mode)
# l = layout([
#     [desc],
#     [inputs, p],
# ], sizing_mode=sizing_mode)

# update()  # initial load of the data

# curdoc().add_root(l)
# curdoc().title = "Movies"


In [24]:
df_noNAN.head()

,X,Y,FGA_perc,FGM_perc
1,-240.0,-17.5,0.435436,35.636364
2,-240.0,2.5,1.174980,40.239643
3,-240.0,22.5,0.678524,51.521225
4,-240.0,42.5,0.394342,46.704212
5,-240.0,62.5,0.258554,30.591402


In [25]:
df_noNAN['plot'] = df_noNAN.FGA_perc

/Users/geoffreypoon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
df_noNAN.head()

,X,Y,FGA_perc,FGM_perc,plot
1,-240.0,-17.5,0.435436,35.636364,0.435436
2,-240.0,2.5,1.174980,40.239643,1.174980
3,-240.0,22.5,0.678524,51.521225,0.678524
4,-240.0,42.5,0.394342,46.704212,0.394342
5,-240.0,62.5,0.258554,30.591402,0.258554


In [38]:
import numpy as np

from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool, Div
from bokeh.models import ColorBar, LinearColorMapper, Plot, Range1d, LinearAxis, FixedTicker, FuncTickFormatter
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import curdoc

output_notebook()

alpha = 0.7

# Create color map
colors = bokeh.palettes.magma(256)[::-1]
mapper = LinearColorMapper(palette = colors, 
                           low=0., high=100.)
ticker = FixedTicker(ticks=[10, 90])
formatter = FuncTickFormatter(code="""
    data = {10: 'low', 90: 'high'}
    return data[tick]
""")
cbar = ColorBar(color_mapper=mapper, ticker=ticker, formatter=formatter,
                major_label_text_align='left',
                major_label_text_font_size='10pt', label_standoff=6,
                border_line_color=None, location=(0, 0),
                orientation="horizontal",
                scale_alpha=alpha
)


# Create Input controls
season = Slider(title="Year season started", start=2004, end=2017, value=2016, step=1)
player = TextInput(title="Player name", value='Stephen Curry')
plot_col = TextInput(title="Color by", value='FGA_perc')


# Create Column Data Source that will be used by the plot
source = ColumnDataSource(data=df_noNAN)

# hover = HoverTool(tooltips=[
#     ('(X, Y)', '(@X, @Y)'),
#     ('FGA% (rel. avg.)', '@FGA_perc%'),
#     ('FG% (rel. avg.)', '@FGM_perc%'),
# ])

hover = HoverTool(
    names = ["heatmap"],
    tooltips=[
                ('(X, Y)', '(@X, @Y)'),
                ('FGA% (rel. avg.)', '@FGA_perc%'),
                ('FG% (rel. avg.)', '@FGM_perc%')
    ]
)

p3 = figure(
        x_range=(-250,250), y_range=(-47.5,470-47.5),
        plot_width=500, plot_height=550,
        tools=[hover], toolbar_location='below'
)
p3.rect(
    x = "X", y = "Y", width=20, height=20,
    source = source,
    fill_color = {'field': 'plot', 'transform': mapper},
    fill_alpha = alpha,
    line_color = None,
    name='heatmap'
)


def select_plot_data():
    season_val = season.value
    player_val = player.value.strip()
    plot_col_val = plot_col.value.strip()
    
    df_temp = df_noNAN.copy()
    df_temp['plot'] = df_noNAN[plot_col_val] * (100./df_noNAN[plot_col_val].max())
    
    info = dict([
        ('season', season_val),
        ('player', player_val),
        ('column', plot_col_val),
    ])
    return df_temp, info


def update():
    print('sup')
    df_plot, info = select_plot_data()

    p3.title.text = "Plotting %s, Season: %d" %(info['column'], info['season'])
    source.data = dict(
        X = df_plot['X'],
        Y = df_plot['Y'],
        plot = df_plot['plot'],
        FGA_perc = df_plot['FGA_perc'],
        FGM_perc = df_plot['FGM_perc']
    )
    
callback = CustomJS(args=dict(source=source), code="""
    console.log('sup');
    source.change.emit();
""")


controls = [season, player, plot_col]
season.js_on_change('value', callback)
#for control in controls:
 #   control.on_change('value', lambda attr, old, new: update())
sizing_mode = 'fixed'  # 'fixed' or scale_width' looks nice with this example

inputs = widgetbox(*controls, sizing_mode=sizing_mode)
l = layout([
    [inputs], [p3]
], sizing_mode=sizing_mode)

update()  # initial load of the data
# curdoc().title = "ShotChart"
show(l)

Loading BokehJS ...

sup


In [ ]:
df_plot.head()

In [ ]:
column = 'FGA_perc'
df_temp = df_noNAN.copy()
df_temp['plot'] = df_noNAN[column] * (100./df_noNAN[column].max())
df_temp.describe()

In [68]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, output_file, show

output_file("js_on_change.html")

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = Figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.change.emit();
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)

In [51]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, output_file, show

output_file("callback.html")

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = Figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

def callback(source=source, window=None):
    data = source.data
    f = cb_obj.value
    x, y = data['x'], data['y']
    for i in range(len(x)):
#         y[i] = window.Math.pow(x[i], f)
    source.change.emit()

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power",
                callback=CustomJS.from_py_func(callback))

layout = column(slider, plot)

show(layout)